In [24]:
import numpy as np
import pandas as pd
import requests
import lxml.html as lh

from tabulate import tabulate
import json, time, datetime
import telegram

pd.set_option('display.max_rows', 500)

In [25]:
# !pip install pandas requests lxml beautifulsoup4 python-telegram-bot tabulate --upgrade

# Telegram API

In [26]:
class TelegramMessenger:
    """
    https://forums.fast.ai/t/training-metrics-as-notifications-on-mobile-using-callbacks/17330/4

    Utilizes this API Library:
       https://github.com/python-telegram-bot/python-telegram-bot
    To install:
       pip install python-telegram-bot --upgrade

    {"api_key": "462203107:<your API key>",
     "chat_id": "<your chat ID>"}

    Here's how you get an API key:
       https://core.telegram.org/api/obtaining_api_id
    Here's how you get your chat ID:
       https://stackoverflow.com/questions/32423837/telegram-bot-how-to-get-a-group-chat-id

    """

    def __init__(self, cred_file_path):
        self.__credentials = json.loads(open(cred_file_path).read())
        # Initialize bot
        self.bot = telegram.Bot(token=self.__credentials['api_key'])

    def send_message(self, message='Done'):
        self.bot.send_message(parse_mode='HTML', chat_id=self.__credentials['chat_id'], text=message)

# Global data

In [86]:
def get_data():
    url = 'https://www.worldometers.info/coronavirus'
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    tr_elements = doc.xpath('//*[@id="main_table_countries_today"]/tbody[1]')
    rows = []
    #For each row, store each first element (header) and an empty list
    for t in tr_elements[0]:
        row = [x.text_content() for x in t.findall('td')[:-1]]
        rows.append(row)

    columns = ['Country', 'Confirmed', 'New', 'Deaths',
               'New Deaths', 'Recovered', 'Active', 'x', 'x']
    df = pd.DataFrame(rows, columns=columns)
    df = df.astype(str).replace('', 0).iloc[:, :-2]
    
    for col in df.columns[1:]:
        df[col] = df[col].str.replace(' ', '').replace(
            ',', '', regex=True).replace('+', '').apply(pd.to_numeric).fillna(0).astype(int)
        
    df = df.sort_values(['New', 'Confirmed'], ascending=False)
    total = df.sum().values
    total[0] = 'Total'
    return pd.concat([pd.DataFrame([total], columns=df.columns), df])
    

df = get_data()
df.head(300)

,Country,Confirmed,New,Deaths,New Deaths,Recovered,Active
0,Total,549359,17549,24872,804,128699,395788
3,Spain,64059,6273,4858,493,9357,49844
4,Germany,47278,3340,281,14,5673,41324
5,Iran,32332,2926,2378,144,11133,18821
11,Belgium,7284,1049,289,69,858,6137
12,Austria,7269,360,58,9,225,6986
18,Israel,3035,342,10,2,79,2946
1,USA,85749,314,1304,9,1868,82577
29,Romania,1292,263,24,1,115,1153
35,Russia,1036,196,3,0,45,988


# India data

In [110]:
def get_time_series():
    url = 'https://api.covid19india.org/data.json'
    page = requests.get(url)
    df = pd.DataFrame(json.loads(page.content)['cases_time_series']).dropna().set_index('date')
    df = df.iloc[-10:-1,:2]
#     df.loc["Total"] = df.sum()
    df.columns = ['Cases', 'Deaths']
    df = df.astype(int)
    df['%Change'] = df.Cases.pct_change()*100
    df = df[-7:]
    df['%Change'] = df['%Change'].astype(int)
    return df
    
get_time_series()

,Cases,Deaths,%Change
date,,,
20 March,59,0,110
21 March,76,0,28
22 March,69,3,-9
23 March,102,2,47
24 March,66,1,-35
25 March,86,1,30
26 March,78,5,-9


In [19]:
def get_data():
    url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSc_2y5N0I67wDU38DjDh35IZSIS30rQf7_NYZhtYYGU1jJYT6_kDx4YpF-qw0LSlGsBYP8pqM_a1Pd/pubhtml#'
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    tr_elements = doc.xpath('//*[@id="1896310216"]/div/table/tbody')
    rows=[]
    #For each row, store each first element (header) and an empty list
    for t in tr_elements[0]:
        row = [x.text_content() for x in t.findall('td')[:-1]]
        rows.append(row)

    df = pd.DataFrame(rows[1:], columns=rows[0]).replace('', None).dropna().drop_duplicates()
    df[['Confirmed', 'Recovered', 'Deaths', 'Active']] = df[['Confirmed', 'Recovered', 'Deaths', 'Active']].apply(pd.to_numeric)
    return df

df = get_data()
print(len(df))
df.head()

38


,State,,Confirmed,Recovered,Deaths,Active
0,Total,,569,40,10,519
2,Kerala,,109,4,0,105
3,Maharashtra,,107,0,2,105
4,Karnataka,,41,3,1,37
5,Telangana,,37,1,0,36


In [92]:
# df_new = df.copy()
# df_new.Confirmed = list(np.random.randint(50, size=10)) + df.Confirmed.to_list()[10:38]
# df_new = df_new.sample(frac=1)
# df_new.head()

In [28]:
def get_clean_table(df):
    message = tabulate(df, headers='keys',
                       tablefmt='simple', numalign="center")
    return '<pre>' + message + '</pre>'

try:
    bot = TelegramMessenger('../india-config.json')
except:
    bot = TelegramMessenger('india-config.json')

In [95]:
curr_date = datetime.datetime.now().date()

while True:
    df_new = get_data()
    df_update = df_new
#     df_update['old_confirmed'] = df.Confirmed
    df_update = df_update.merge(df.rename({'Confirmed': 'old_confirmed'}, axis=1))
    df_update['New'] = df_update['Confirmed'] - df_update['old_confirmed']
    df_update = df_update[df_update['New']>0]
    print(df_update)
    df_update = df_update[['State', 'New', 'Confirmed', 'Deaths']].set_index('State')
    curr_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    curr_time = f'Case update at: {curr_time}'
    print(curr_time)
    if len(df_update)>0:
        message = get_clean_table(df_update)
        print(message)
        bot.send_message(curr_time)
        bot.send_message(message)
    else:
        print('No new cases')
    date = datetime.datetime.now().date()
    if date != curr_date:
        df = df_new
#     break
    time.sleep(600)

Empty DataFrame
Columns: [State, , Confirmed, Recovered, Deaths, Active, old_confirmed, New]
Index: []
Case update at: 2020-03-23 23:31
No new cases


KeyboardInterrupt: 

In [393]:
# !jupyter nbconvert --to script telegram.ipynb

[NbConvertApp] Converting notebook telegram.ipynb to script
[NbConvertApp] Writing 3206 bytes to telegram.py


In [111]:
bot = TelegramMessenger('../test-config.json')

In [9]:
df = get_data()

In [10]:
df = df[['Country', 'New', 'Confirmed', 'Deaths']]
df.Country = df.Country.apply(lambda x: x[:8])
df = df.rename({'Country': 'Count','Confirmed':'Conf'}, axis=1).set_index('Count')

In [18]:
# .replace(' | ', '|').replace('------|', '|')
message = '<pre>' + tabulate(df[:40], headers='keys', tablefmt='simple', numalign="center") + '</pre>'
bot.send_message(message)

In [76]:
# url = f'https://api.telegram.org/bot11352/getUpdates'
# requests.get(url).text

In [33]:
# for g, sub_df in df.groupby(np.arange(len(df)) // 40):
#     print(sub_df.shape)

In [112]:
bot.send_message(get_clean_table(get_time_series()))